# ML for Bioinformatics (30 points)
## HW5 - Long short-term memory

---

Name: 

Student No.:

---


# ECG Heartbeat Classification

In this exercise we aim to perform a classification task on a bunch of ECG recorded signals. To this end, you will implement two models with different architectures: 1- LSTM neural network 2- Convolutional neural network. You are supposed to achieve a high performance on both models.

Attention: In this notebook you are only allowed to change the sections denoted by the following mark:

    # ========================= Place Your Code Here ========================= #

    # ======================================================================== #



# Collect Data
We use a preprocessed version of a famous datasets in heartbeat classification, [the MIT-BIH Arrhythmia Dataset](https://physionet.org/content/mitdb/1.0.0/). 
The signals in this dataset correspond to electrocardiogram (ECG) shapes of heartbeats for the normal case and the cases affected by different arrhythmias and myocardial infarction. We will use the Arrhythmia Dataset used in [this paper](https://arxiv.org/abs/1805.00794) which is the preprocessed arrhythmia dataset consisting of signals that are preprocessed and segmented, with each segment corresponding to a heartbeat in the dataset. This dataset is composed of 109446 samples which are classified into 5 categories.

You must download the dataset from [here](https://drive.google.com/file/d/1BhedBaKpVPT_QJgGFPIc-UiCj_PhoOwo/view?usp=sharing). It is also accessible on [kaggle](https://www.kaggle.com/shayanfazeli/heartbeat).
The training and test data are located in mitbih_train.csv and mitbih_test.csv files. Locate these files in dataset folder next to this notebook. If you are running this notebook on colab (which is strongly recommended), you should first copy the dataset into your Google drive and the then running the code bellow will do the rest.

In [ ]:
FOLDERNAME = None

In [ ]:
assert FOLDERNAME is not None, "[!] Enter the foldername."

In [ ]:
from google.colab import drive

drive.mount('/content/drive', force_remount=True)

%cp -r /content/drive/MyDrive/$FOLDERNAME /content/
!unzip /content/$FOLDERNAME/29414_37484_bundle_archive.zip



In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from tqdm.auto import tqdm, trange
from sklearn.metrics import roc_auc_score
from sklearn.metrics import classification_report
from IPython import display
import torch.backends.cudnn as cudnn
import random

def set_seed(seed):
    cudnn.deterministic = True
    torch.cuda.manual_seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    random.seed(seed)

In [ ]:
if torch.cuda.is_available():
    device = torch.device("cuda:0")
    print("Running on the GPU")
else:
    device = torch.device("cpu")
    print("Running on the CPU")

# Read The Dataset (3 points)
This dataset consists of a series of CSV files. Each of these CSV files contain a matrix, with each row representing an example in that portion of the dataset. The final element of each row denotes the class to which that example belongs. In this section you are asked to load the samples from "mitbih_train.csv" and "mitbih_test.csv" files into seperate numpy arrays. 

hint: you can use pandas.read_csv function for reading csv files.

In [ ]:
# ========================= Place Your Code Here ========================= #

dataframe_train = ??
dataframe_test = ??

data_train = ??
labels_train = ??

data_test = ??
labels_test = ??

# ======================================================================== #

Let's check your code. You may be able to pass the following tests if the data is loaded correctly.

In [ ]:

assert data_train.shape == (87554, 187)
assert labels_train.shape == (87554,)
assert len (labels_train[labels_train == 4]) == 6431
assert data_test.shape == (21892, 187)
assert labels_test.shape == (21892,)

assert data_train.dtype == np.float32
assert labels_train.dtype == np.int64
assert data_test.dtype  == np.float32
assert labels_test.dtype  == np.int64
print ("Test passed")

Now let's see an example of the signals in the dataset. By running the code bellow you should be able to see a graph containing two plots with labels 1 and 4.

In [ ]:
t = np.linspace (0,(187/125)*1000, 187)
plt.plot (t,data_train[labels_train == 1][0] , label = f"label:{labels_train[labels_train == 1][0]}")
plt.plot (t,data_train[labels_train == 4][0] , label = f"label:{labels_train[labels_train == 4][0]}")
plt.legend()
plt.xlabel("time (ms)")
plt.show()

# Create Custom Dataset (3 points)
You need to create your custom dataset so you can define a dataloader to use in the train and test phase. Your class must implement init, getitem and len methods.

In [ ]:
class HeartbeatDataset(Dataset):
    
    def __init__(self, data, labels):
        # ========================= Place Your Code Here ========================= #
        pass
        # ======================================================================== #
        
    def __getitem__(self, index):
        # ========================= Place Your Code Here ========================= #
        pass
        # ======================================================================== #        
        
        
    def __len__(self):
        # ========================= Place Your Code Here ========================= #
        pass
        # ======================================================================== #                


batch_size = 64

train_set = HeartbeatDataset(data_train, labels_train)
train_loader = DataLoader(dataset=train_set,
                          batch_size=batch_size,
                          shuffle=True)

test_set = HeartbeatDataset(data_test, labels_test)
test_loader = DataLoader(dataset=test_set,
                        batch_size=batch_size,
                         shuffle=True)



Make sure that the following test passes.

In [ ]:
assert len (train_set) == 87554
assert len (test_set) == 21892
(batch_data, batch_label) = next (iter (train_loader))
assert batch_data.shape == (batch_size, 187)
assert batch_label.shape == (batch_size,)
print ("passed")

In this part we will find the number of samples in each class. We are going to need this in next sections.

In [ ]:
num_classes = 5
class_samples_num = [0 for i in range(num_classes)]
for data in train_loader:
    inputs, labels = data        
    for i in labels:
        class_samples_num[i] += 1

print(class_samples_num)

# Create LSTM Classifier (8 points)
Now you are ready to implement your LSTM neural network which inherits from nn.Module. The structure of the network must be as follows:

  (lstm): LSTM(1, 64, batch_first=True)<br>
  (fc1): Linear(in_features=64, out_features=32, bias=True)<br>
  (relu): ReLU()<br>
  (fc2): Linear(in_features=32, out_features=5, bias=True)<br>

At each step the value of signal is given as input to the LSTM block. The hidden state and cell state get updated. Finally the output is computed by two fully connected on top of hidden states with a softmax layer at the end.

Attention: Be carefull about tensor shapes, Dont forget to unsqueeze the input.

In [ ]:
class LSTMClassifier(nn.Module):

    def __init__(self):
        super(LSTMClassifier, self).__init__() 
        # ========================= Place Your Code Here ========================= #
        pass
        # ======================================================================== #   

    def forward(self, x):
        # ========================= Place Your Code Here ========================= #
        pass
        # ======================================================================== #   


Now let's check if your output shape is correct:

In [ ]:
(batch_data, batch_label) = next (iter (train_loader))
model = LSTMClassifier().to(device)
output = model (batch_data.to(device)).cpu()
assert output.shape == (batch_size, 5)
print ("passed")

# Define loss function and optimizer (2 points)
Next you can instance a classifier, loss function and optimizer. Because of the imbalanced dataset, the model may tend to label the whole test data as "normal". So we must make the model pay more attention to the minor cases. There are different ways to do so. One of them is the upsampling technique in which we produce enough samples for all categories, by resampling from the present data. Another way is to weight loss of each sample proportional to inverse of its caregory's number of samples. So weights of samples of each category sum to one, and the model pays equal attention to all categories. We are going to use this technique by providing the CrossEntropyLoss loss function an array of weights (You can also use other techniques like upsampling). Make sure to visit the [PyTorch page](https://pytorch.org/docs/stable/generated/torch.nn.CrossEntropyLoss.html) on cross entropy loss. 

In [ ]:
lstm_classifier = LSTMClassifier()
lstm_classifier.to(device)

def get_loss_weights(class_samples_num):
    # ========================= Place Your Code Here ========================= #
    loss_weight = ??
    # ======================================================================== #   
    return loss_weight

criterion = nn.CrossEntropyLoss(weight=get_loss_weights(class_samples_num))
criterion.to(device)
optimizer = torch.optim.Adam(lstm_classifier.parameters() , lr=1e-3)

In [ ]:
data , label = next(iter(train_loader))
output = lstm_classifier(data.to(device))

assert output.shape == (batch_size, 5) 

# Train the classifier
The following code runs the training phase using what you have build so far. It also feeds the test data to the model at each epoch without updating the weights, so we can study the loss on both training and test data during the training phase by plotting its diagram at the end. You are free to change the number of epochs to acheive the best soloution.

In [ ]:
# ========================= Place Your Code Here ========================= #
epoch_num = 30
# ======================================================================== #   

train_log = []
test_log = []
set_seed(111)
for epoch in range(1, epoch_num+1):
    
    running_loss = 0    
    train_loss = []
    lstm_classifier.train()
    for (inputs, labels) in tqdm(train_loader, desc='Training epoch ' + str(epoch), leave=False):        
        inputs, labels = inputs.to(device), labels.to(device)        
        optimizer.zero_grad()
        outputs = lstm_classifier(inputs)          
        loss = criterion(outputs, labels)        
        loss.backward()                
        optimizer.step()        
        train_loss.append(loss.item())
    train_log.append(np.mean(train_loss))

    running_loss = 0
    test_loss = []
    lstm_classifier.eval()
    with torch.no_grad():                
        for (inputs, labels) in tqdm(test_loader, desc='Test', leave=False):         
            inputs, labels = inputs.to(device), labels.to(device)        
            outputs = lstm_classifier(inputs)                       
            loss = criterion(outputs, labels)            
            test_loss.append(loss.item())
    test_log.append(np.mean(test_loss))    
    plt.plot(range(1, epoch+1), train_log, color='C0')
    plt.plot(range(1, epoch+1), test_log, color='C1')
    display.clear_output(wait=True)
    display.display(plt.gcf())

# Test the classifier
The following code runs the LSTM classifier on test data and reports the [classification report](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.classification_report.html). It also reports the [ROC AUC score](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.roc_auc_score.html) which shows us how much we have succeeded in handling the imbalanced data. The ROC AUC score is the evaluation criterion of your model. ROC AUC score and accuracy of more than 0.9 are achievable by the model. You are supposed to achieve ROC AUC score and accuracy of at least 0.8.

In [ ]:
y_true = []
y_pred = []
with torch.no_grad():
    lstm_classifier.eval()
    for (inputs, labels) in tqdm(test_loader, desc='Test'):         
        inputs, labels = inputs.to(device), labels.to(device)        
        outputs = lstm_classifier(inputs)                       
        y_true += torch.eye(num_classes)[labels].tolist()
        y_pred += outputs.tolist()
y_true = np.array(y_true)
y_pred = np.array(y_pred)
print('ROC_AUC Score:', roc_auc_score(y_true, y_pred))
print(classification_report(y_true.argmax(axis=1), y_pred.argmax(axis=1)))

# Create Convolutional Classifier (8 points)
Next we are going to assemble a convolution neural netwok. Use the following blocks: <br>
  (conv1): Conv1d(cin = 1, cout = 64, kernel size = 19)<br>
  (fc1): Linear(in_features=64*187, out_features=32, bias=True)<br>
  (relu): ReLU()<br>
  (fc2): Linear(in_features=32, out_features=5, bias=True)<br>

  Again in the same way fill the placeholders and train the conv model.


In [ ]:
class ConvClassifier(nn.Module):

    def __init__(self):
        super(ConvClassifier, self).__init__()
        # ========================= Place Your Code Here ========================= #
        pass
        # ======================================================================== #   

    def forward(self, x):
        # ========================= Place Your Code Here ========================= #
        pass
        # ======================================================================== #           


In [ ]:
conv_classifier = ConvClassifier()
conv_classifier.to(device)

criterion = nn.CrossEntropyLoss(weight = get_loss_weights(class_samples_num))
criterion.to(device)
optimizer = torch.optim.Adam(conv_classifier.parameters())


In [ ]:
data , label = next(iter(train_loader))
output = conv_classifier(data.to(device))

assert output.shape == (batch_size, 5) 

In [ ]:
# ========================= Place Your Code Here ========================= #
epoch_num = 30
# ======================================================================== #   

train_log = []
test_log = []
set_seed(111)
for epoch in range(1, epoch_num+1):
    
    running_loss = 0    
    train_loss = []
    conv_classifier.train()
    for (inputs, labels) in tqdm(train_loader, desc='Training epoch ' + str(epoch), leave=False):        
        inputs, labels = inputs.to(device), labels.to(device)        
        optimizer.zero_grad()
        outputs = conv_classifier(inputs)          
        loss = criterion(outputs, labels)        
        loss.backward()                
        optimizer.step()        
        train_loss.append(loss.item())
    train_log.append(np.mean(train_loss))

    running_loss = 0
    test_loss = []
    conv_classifier.eval()
    with torch.no_grad():                
        for (inputs, labels) in tqdm(test_loader, desc='Test', leave=False):         
            inputs, labels = inputs.to(device), labels.to(device)        
            outputs = conv_classifier(inputs)                       
            loss = criterion(outputs, labels)            
            test_loss.append(loss.item())
    test_log.append(np.mean(test_loss))    
    plt.plot(range(1, epoch+1), train_log, color='C0')
    plt.plot(range(1, epoch+1), test_log, color='C1')
    display.clear_output(wait=True)
    display.display(plt.gcf())

In [ ]:
y_true = []
y_pred = []
with torch.no_grad():
    conv_classifier.eval()
    for (inputs, labels) in tqdm(test_loader, desc='Test'):         
        inputs, labels = inputs.to(device), labels.to(device)        
        outputs = conv_classifier(inputs)                       
        y_true += torch.eye(num_classes)[labels].tolist()
        y_pred += outputs.tolist()
y_true = np.array(y_true)
y_pred = np.array(y_pred)
print('ROC_AUC Score:', roc_auc_score(y_true, y_pred))
print(classification_report(y_true.argmax(axis=1), y_pred.argmax(axis=1)))

# Comparing The Models (6 points)

In [ ]:
def count_parameters(model): 
  return sum(p.numel() for p in model.parameters() if p.requires_grad)

Let's first compare the number of parameters:

In [ ]:
print ("Parameters for lstm classifier=" , count_parameters(lstm_classifier))

In [ ]:
print ("Parameters for conv classifier=" , count_parameters(conv_classifier))

Based on your observations answer the following questions:

1. Which model has more parameters? why?
****** your answer here **********

2. Which model acheived better accuracy? why?
****** your answer here **********

3. Is it always possible to apply conv networks to sequences? Explain when we can. What about LSTMS?
****** your answer here **********

# Useful Links and Acknowledgements

[An Effective LSTM Recurrent Network to Detect Arrhythmia on Imbalanced ECG Dataset](https://www.hindawi.com/journals/jhe/2019/6320651/)

[Classify ECG Signals Using Long Short-Term Memory Networks
](https://https://www.mathworks.com/help/signal/examples/classify-ecg-signals-using-long-short-term-memory-networks.html)

[Sequence Models and Long-Short Term Memory Networks](https://https://pytorch.org/tutorials/beginner/nlp/sequence_models_tutorial.html)

[LSTMs for Time Series in PyTorch](https://www.jessicayung.com/lstms-for-time-series-in-pytorch/)

[ECG Heartbeat Classification: A Deep Transferable Representation
](https://https://arxiv.org/abs/1805.00794)

[MIT-BIH Arrhythmia Database](https://https://physionet.org/content/mitdb/1.0.0/)